# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [39]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url	slides_url
2025-01-01	Optimization of Energy Efficiency for Federated Learning over IRS-assisted Cell-free Massive MIMO Networks	IEEE Transactions on Wireless Communications	This paper explores the optimization of energy efficiency for federated learning in an intelligent reflecting surface (IRS)-assisted cell-free massive MIMO network.	Dang, X.-T., Nguyen, H. V., & Shin, O.-S. (2025). "Optimization of Energy Efficiency for Federated Learning over IRS-assisted Cell-free Massive MIMO Networks." <i>IEEE Transactions on Wireless Communications</i>.	optimization-energy-efficiency-federated-learning-irs-mimo	https://doi.org/10.1109/TWC.2025.3588353	#slides-not-available
2025-01-01	Optimizing Energy Efficiency for Federated Learning in Rotary-Wing UAV Air-to-Ground Communications	IEEE Transactions on Green Communications and Networking	Focuses on enhancing the energy efficiency of federated learning systems utilizing rotary-wing unmann

## Import pandas

We are using the very handy pandas library for dataframes.

In [40]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [41]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url,slides_url
0,2025-01-01,Optimization of Energy Efficiency for Federate...,IEEE Transactions on Wireless Communications,This paper explores the optimization of energy...,"Dang, X.-T., Nguyen, H. V., & Shin, O.-S. (202...",optimization-energy-efficiency-federated-learn...,https://doi.org/10.1109/TWC.2025.3588353,#slides-not-available
1,2025-01-01,Optimizing Energy Efficiency for Federated Lea...,IEEE Transactions on Green Communications and ...,Focuses on enhancing the energy efficiency of ...,"Dang, X.-T., Nguyen, Q.-S., & Shin, O.-S. (202...",optimizing-energy-efficiency-federated-learnin...,https://doi.org/10.1109/TGCN.2025.3599309,#slides-not-available
2,2025-08-01,A DDPG-LSTM Framework for Optimizing UAV-Enabl...,Drones,Presents a framework combining Deep Determinis...,"Dang, X.-T., Eom, J.-S., Vu, B.-M., & Shin, O....",ddpg-lstm-framework-optimizing-uav-sensing-com...,https://doi.org/10.3390/drones9080548,#slides-not-available
3,2025-04-22,Combining the Viterbi Algorithm and Graph Neur...,Electronics,Investigates a novel method for efficient Mult...,"Nguyen, T. A., Dang, X.-T., Shin, O.-S., & Lee...",viterbi-graph-neural-networks-mimo-detection,https://doi.org/10.3390/electronics14091698,#slides-not-available
4,2024-01-01,Physical Layer Security for IRS-UAV-Assisted C...,IEEE Access,Analyzes and enhances physical layer security ...,"Dang, X.-T., Nguyen, H. V., & Shin, O.-S. (202...",physical-layer-security-irs-uav-massive-mimo,https://doi.org/10.1109/ACCESS.2024.3419888,#slides-not-available
5,2024-12-23,A Survey on Energy-Efficient Design for Federa...,Energies,A comprehensive review detailing the current s...,"Dang, X.-T., Vu, B.-M., Nguyen, Q.-S., Tran, T...",survey-energy-efficient-federated-learning-wir...,https://doi.org/10.3390/en17246485,#slides-not-available
6,2024-05-08,Optimization of Energy Efficiency for Federate...,Electronics,Focuses on optimizing the energy efficiency of...,"Dang, X.-T., & Shin, O.-S. (2024). ""Optimizati...",optimization-energy-efficiency-federated-learn...,https://doi.org/10.3390/electronics13101827,#slides-not-available
7,2023-01-01,Optimization of IRS-NOMA-Assisted Cell-Free Ma...,IEEE Access,Applies Deep Reinforcement Learning (DRL) to o...,"Dang, X.-T., Nguyen, H. V., & Shin, O.-S. (202...",optimization-irs-noma-mimo-deep-rl,https://doi.org/10.1109/access.2023.3310283,#slides-not-available
8,2023-04-01,Optimal User Pairing Approach for NOMA-Based C...,IEEE Transactions on Vehicular Technology,Proposes an optimal user pairing approach for ...,"Dang, X.-T., Le, M. T. P., Nguyen, H. V., Chat...",optimal-user-pairing-noma-cell-free-massive-mimo,https://doi.org/10.1109/TVT.2022.3225835,#slides-not-available


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [42]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [43]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.slides_url)) > 5:
        md += "\nslidesurl: '" + item.slides_url + "'"

    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\n Cite: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"

    if len(str(item.slides_url)) > 5:
        md += "\n[Download slides here](" + item.slides_url + ")\n" 

    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\n Cite: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [44]:
!ls ../_publications/

2009-10-01-paper-title-number-1.md
2010-10-01-paper-title-number-2.md
2015-10-01-paper-title-number-3.md
2023-01-01-optimization-irs-noma-mimo-deep-rl.md
2023-04-01-optimal-user-pairing-noma-cell-free-massive-mimo.md
2024-01-01-physical-layer-security-irs-uav-massive-mimo.md
2024-02-17-paper-title-number-4.md
2024-05-08-optimization-energy-efficiency-federated-learning-uav-networks.md
2024-12-23-survey-energy-efficient-federated-learning-wireless.md
2025-01-01-optimization-energy-efficiency-federated-learning-irs-mimo.md
2025-01-01-optimizing-energy-efficiency-federated-learning-uav-communications.md
2025-04-22-viterbi-graph-neural-networks-mimo-detection.md
2025-06-08-paper-title-number-5.md
2025-08-01-ddpg-lstm-framework-optimizing-uav-sensing-communication.md


In [45]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

---
title: "Paper Title Number 1"
collection: publications
category: manuscripts
permalink: /publication/2009-10-01-paper-title-number-1
excerpt: 'This paper is about the number 1. The number 2 is left for future work.'
date: 2009-10-01
venue: 'Journal 1'
slidesurl: 'https://academicpages.github.io/files/slides1.pdf'
paperurl: 'https://academicpages.github.io/files/paper1.pdf'
bibtexurl: 'https://academicpages.github.io/files/bibtex1.bib'
citation: 'Your Name, You. (2009). &quot;Paper Title Number 1.&quot; <i>Journal 1</i>. 1(1).'
---
The contents above will be part of a list of publications, if the user clicks the link for the publication than the contents of section will be rendered as a full page, allowing you to provide more information about the paper for the reader. When publications are displayed as a single page, the contents of the above "citation" field will automatically be included below this section in a smaller font.
